<a href="https://colab.research.google.com/github/yananma/5_programs_per_day/blob/master/02145.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 6.8 长短期记忆 ( LSTM )

### 6.8.2 读取数据

In [0]:
import numpy as np 
import torch 
from torch import nn, optim 
import torch.nn.functional as F 
import d2l 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
!mkdir ../../data

In [5]:
!git clone https://github.com/ShusenTang/Dive-into-DL-PyTorch

Cloning into 'Dive-into-DL-PyTorch'...
remote: Enumerating objects: 1692, done.
remote: Total 1692 (delta 0), reused 0 (delta 0), pack-reused 1692
Receiving objects: 100% (1692/1692), 25.29 MiB | 5.02 MiB/s, done.
Resolving deltas: 100% (975/975), done.


In [0]:
!cp Dive-into-DL-PyTorch/data/jaychou_lyrics.txt.zip ../../data

In [0]:
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

### 6.8.3 从零开始实现

#### 1 初始化模型参数

In [15]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size 
print('will use', device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    def _three():
        return (_one((num_inputs, num_hiddens)), 
            _one((num_hiddens, num_hiddens)), 
            torch.nn.Parameter(torch.zeros(num_hiddens, device=device, dtype=torch.float32), requires_grad=True))

    W_xi, W_hi, b_i = _three()
    W_xf, W_hf, b_f = _three()
    W_xo, W_ho, b_o = _three()
    W_xc, W_hc, b_c = _three()

    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, dtype=torch.float32), requires_grad=True)
    return nn.ParameterList([W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q])

will use cuda


#### 2 定义模型

In [0]:
def init_lstm_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), 
        torch.zeros((batch_size, num_hiddens), device=device))

In [0]:
def lstm(inputs, state, params):
    [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q] = params 
    (H, C) = state 
    outputs = []
    for X in inputs:
        I = torch.sigmoid(torch.matmul(X, W_xi) + torch.matmul(H, W_hi) + b_i)
        F = torch.sigmoid(torch.matmul(X, W_xf) + torch.matmul(H, W_hf) + b_f)
        O = torch.sigmoid(torch.matmul(X, W_xo) + torch.matmul(H, W_ho) + b_o)
        C_tilda = torch.tanh(torch.matmul(X, W_xc) + torch.matmul(H, W_hc) + b_c)
        C = F * C + I * C_tilda 
        H = O * C.tanh()
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H, C)

#### 3 训练模型并创作歌词

In [0]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

In [22]:
d2l.train_and_predict_rnn(lstm, get_params, init_lstm_state, num_hiddens, vocab_size, device, corpus_indices, idx_to_char,
            char_to_idx, False, num_epochs, num_steps, lr, clipping_theta, batch_size, pred_period, pred_len, 
            prefixes)

epoch 40, perplexity 213.371680, time 0.60 sec
 - 分开 我不的 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 
 - 不分开 我不的 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 
epoch 80, perplexity 65.394913, time 0.60 sec
 - 分开 我想我这你的你 我想 你想你想你 我想 我想你的爱我 你不 我想我想你的你 爱爱我 你你我 我不要
 - 不分开 你不我 你不我 我不要 我不要 我不我 你不我 我不要 我不我 你不我 我不要 我不我 你不我 我
epoch 120, perplexity 15.610854, time 0.60 sec
 - 分开 我想你这你 我不要这样 我不 我不 我不能 爱情走的太快就像龙卷风 不不能能我 我不要再想 我不 
 - 不分开 你这你的生笑 一天  说你的话笑 一通  说你的话笑 快你在对我的你的我 我想揍你已你 这这不这样
epoch 160, perplexity 3.942796, time 0.61 sec
 - 分开 我说带你 我想多难熬  没有你在我有多难熬多烦恼  没有你烦 我有著努恼 你有 我不我的爱笑 我爱
 - 不分开你的让 我想要你已你很你 想要再你样打我妈妈 我道的你 你甘著听 不要再这样我妈妈 我想你的你甘你 


### 6.8.4 简洁实现

In [24]:
lr = 1e-2 
lstm_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens)
model = d2l.RNNModel(lstm_layer, vocab_size)
d2l.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device, corpus_indices, idx_to_char, char_to_idx, 
                num_epochs, num_steps, lr, clipping_theta, batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 1.019573, time 0.15 sec
 - 分开始移动 回到当初爱你的时空 停格内容不忠 所有回忆对着我进攻 我的伤口被你拆封 誓言太沉重泪被纵容 
 - 不分开 我不能 爱情走的太快就像龙卷风 不能承受我已无处可躲 我不要再想 我不要再想 我不 我不 我不要再
epoch 80, perplexity 1.012941, time 0.15 sec
 - 分开始移动 回到当初爱你的时空 停格内容不忠 所有回忆对着我进攻 我的伤口被你拆封 誓言太沉重泪被纵容 
 - 不分开 我不到 整个人对就怎么小 就怎么每天祈祷我的心跳你知道  杵在伊斯坦堡 却只想你和汉堡 我想要你的
epoch 120, perplexity 1.012826, time 0.15 sec
 - 分开始移动 可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 漂亮的让我面红的可爱女人 温柔
 - 不分开 我不到 整颗心悬在半空 我只能够远远看著 这些我都做得到 但那个人已经不是我 没有你在 我有多难熬
epoch 160, perplexity 1.012320, time 0.15 sec
 - 分开始移动 回到当初爱你的时空 停格内容不忠 所有回忆对着我进攻 我的伤口被你拆封 誓言太沉重泪被纵容 
 - 不分开 我不到 爱情我的太快就像龙卷风 不能承受我已无处可躲 我不要再想 我不要再想 我不 我不 我不要再
